In [1]:
from gensim.models.doc2vec import *

In [2]:
model = Doc2Vec.load('../store/doc2vec_model_800_beer_df10.mod')

In [4]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
beer_data = cPickle.load(open('../../beer_data/beer_vec_ds_df10.p', 'rb'))

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

idxs = val_idxs
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
X = beer_data.X[val_idxs]
X = map(lambda s : [beer_data.idx2word[t] for t in s if t > 0], X)

In [7]:
embedds = []
for sent in X :
    embedds.append(model.infer_vector(sent))

In [8]:
import numpy as np
embedds = np.array(embedds)
embedds.shape

(9121, 800)

In [9]:
from sklearn.preprocessing import normalize
embedds_n = normalize(embedds, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [18]:
from sklearn.metrics import roc_auc_score

for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.631513528458
0 0.62307294114
3 0.633458764682
2 0.632344799721


In [10]:
from sklearn.metrics import roc_auc_score

for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.612856138708
0 0.608076261421
3 0.61453558881
2 0.614228415827


In [36]:
H['5'] = np.zeros((len(idxs), len(idxs)))
a0 = set(ds[ds['review/overall'] < 3].index) & set(idxs)
a1 = set(ds[ds['review/overall'] > 3].index) & set(idxs)
a0 = map(lambda s : list(idxs).index(s), a0)
a1 = map(lambda s : list(idxs).index(s), a1)
for j in a0 :
    H['5'][j, a0] = 1
for j in a1 :
    H['5'][j, a1] = 1

H['5'][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [43]:
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H['5'][i], scores[i])
print np.mean(aucs)

0.630622114469
